**Classifier**

In [4]:
# =============================================================================
# [Stage 1] 노이즈 분류기 학습 (KoELECTRA)
# =============================================================================
!pip install transformers torch pandas tqdm scikit-learn

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from google.colab import drive  # <--- [수정] drive 모듈을 명시적으로 임포트

drive.mount('/content/drive')

# GPU 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. 데이터 로드 및 라벨링
# Noise 1 (일반/오타) = 0
# Noise 2 (야민정음) = 1

path1 = '/content/drive/MyDrive/P01_Nandoc/dataset/일반_10k.csv'
path2 = '/content/drive/MyDrive/P01_Nandoc/dataset/yamin_11k.csv'
path3 = '/content/drive/MyDrive/P01_Nandoc/dataset/new_train_yamin_only.csv'
path4 = '/content/drive/MyDrive/P01_Nandoc/dataset/new_train_normal_only.csv'

# 데이터 로드
df1 = pd.read_csv(path1)[['input']].dropna() # 일반 10k
df2 = pd.read_csv(path2)[['input']].dropna() # 야민 11k
df3 = pd.read_csv(path3)[['input']].dropna() # 야민 new
df4 = pd.read_csv(path4)[['input']].dropna() # 일반 new

# 라벨링
df1['label'] = 0
df4['label'] = 0
df2['label'] = 1
df3['label'] = 1

# Noise 1 (일반) 통합 및 15k 샘플링
df_noise1 = pd.concat([df1, df4])
if len(df_noise1) > 15000:
    df_noise1 = df_noise1.sample(n=15000, random_state=42).reset_index(drop=True)
else:
    df_noise1 = df_noise1.reset_index(drop=True)

# Noise 2 (야민정음) 통합 및 15k 샘플링
df_noise2 = pd.concat([df2, df3])
if len(df_noise2) > 15000:
    df_noise2 = df_noise2.sample(n=15000, random_state=42).reset_index(drop=True)
else:
    df_noise2 = df_noise2.reset_index(drop=True)

# 데이터 통합 및 셔플
df_classifier = pd.concat([df_noise1, df_noise2]).sample(frac=1, random_state=42).reset_index(drop=True)
train_df, val_df = train_test_split(df_classifier, test_size=0.1, random_state=42)

print(f"✅ 분류기 학습 데이터: {len(train_df)}개 (Type 0: {sum(train_df['label']==0)}, Type 1: {sum(train_df['label']==1)})")
print(f"✅ 분류기 검증 데이터: {len(val_df)}개 (Type 0: {sum(val_df['label']==0)}, Type 1: {sum(val_df['label']==1)})")

# 2. 모델 & 토크나이저 설정
MODEL_NAME = "monologg/koelectra-small-v3-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

# 3. 데이터셋 클래스
class NoiseTypeDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = str(row['input'])
        label = int(row['label'])

        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_ds = NoiseTypeDataset(train_df, tokenizer)
val_ds = NoiseTypeDataset(val_df, tokenizer)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

# 4. 학습 (3 Epoch)
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 3

print("🚦 노이즈 분류기(Classifier) 학습 시작...")
for epoch in range(epochs):
    model.train()
    total_acc = 0
    total_loss = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 정확도 계산
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        acc = (predictions == labels).sum().item() / len(labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += acc
        loop.set_postfix(loss=loss.item(), acc=acc)

    print(f"Epoch {epoch+1} Avg Acc: {total_acc / len(train_loader):.4f}")

# 5. 분류기 저장
model.save_pretrained("/content/drive/MyDrive/P01_Nandoc/Classifier")
tokenizer.save_pretrained("/content/drive/MyDrive/P01_Nandoc/Classifier")
print("💾 분류기 저장 완료: /content/drive/MyDrive/P01_Nandoc/Classifier")

Mounted at /content/drive
✅ 분류기 학습 데이터: 27000개 (Type 0: 13524, Type 1: 13476)
✅ 분류기 검증 데이터: 3000개 (Type 0: 1476, Type 1: 1524)


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/56.6M [00:00<?, ?B/s]

...

Epoch 1 Avg Acc: 0.9827


Epoch 2:   0%|          | 0/422 [00:00<?, ?it/s]

Epoch 2 Avg Acc: 0.9950


Epoch 3:   0%|          | 0/422 [00:00<?, ?it/s]

Epoch 3 Avg Acc: 0.9962
💾 분류기 저장 완료: /content/drive/MyDrive/P01_Nandoc/Classifier


**엔트로피 기반 성능 평가**

In [5]:
# =============================================================================
# [Stage 2] 엔트로피(Entropy) 기반 성능 평가 및 분석
# =============================================================================
import torch.nn.functional as F
import numpy as np

print("\n📊 엔트로피(Entropy) 기반 평가 시작...")

model.eval()
all_entropies = []
all_preds = []
all_labels = []
all_texts = []
correct_mask = []

# 시각화를 위해 원본 텍스트가 필요하므로, decode를 수행합니다.
print("   -> 검증 데이터셋에 대한 예측 및 엔트로피 계산 중...")

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 모델 예측
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # 1. 확률값 계산 (Softmax)
        probs = F.softmax(logits, dim=-1)

        # 2. 엔트로피 계산: -sum(p * log(p))
        # (log(0) 방지를 위해 아주 작은 값 1e-9를 더해줌)
        entropy = -torch.sum(probs * torch.log(probs + 1e-9), dim=-1)

        # 3. 예측값 추출
        preds = torch.argmax(logits, dim=-1)

        # 4. 결과 저장
        all_entropies.extend(entropy.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # 정답 여부 저장 (True/False)
        correct_mask.extend((preds == labels).cpu().numpy())

        # 텍스트 디코딩 (입력 문장 복원)
        # batch_size만큼 반복
        for i in range(input_ids.size(0)):
            decoded_text = tokenizer.decode(input_ids[i], skip_special_tokens=True)
            all_texts.append(decoded_text)

# numpy 배열로 변환
all_entropies = np.array(all_entropies)
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
correct_mask = np.array(correct_mask)

# -----------------------------------------------------------------------------
# [결과 분석 1] 정답 vs 오답 엔트로피 비교
# -----------------------------------------------------------------------------
avg_entropy_correct = np.mean(all_entropies[correct_mask])
avg_entropy_wrong = np.mean(all_entropies[~correct_mask])

print("\n" + "="*50)
print("🧐 [분석 결과 1] 모델의 확신도(Confidence) 분석")
print("="*50)
print(f"✅ 정답을 맞춘 경우 평균 엔트로피 : {avg_entropy_correct:.4f} (낮을수록 확신함)")
if len(all_entropies[~correct_mask]) > 0:
    print(f"❌ 정답을 틀린 경우 평균 엔트로피 : {avg_entropy_wrong:.4f} (높을수록 헷갈려함)")
else:
    print("❌ 틀린 데이터가 없습니다! (완벽한 모델)")

print("\n👉 해석: 일반적으로 '틀린 경우'의 엔트로피가 '맞춘 경우'보다 높아야 모델이 모르는 것을 모른다고 인식하는 정상적인 상태입니다.")


# -----------------------------------------------------------------------------
# [결과 분석 2] 가장 헷갈려했던(High Entropy) 샘플 Top 5
# -----------------------------------------------------------------------------
# 엔트로피가 높은 순서대로 정렬 인덱스 가져오기
top_k = 5
sorted_indices = np.argsort(all_entropies)[::-1] # 내림차순 정렬
top_indices = sorted_indices[:top_k]

print("\n" + "="*50)
print(f"🤯 [분석 결과 2] 모델이 가장 헷갈려한 문장 Top {top_k}")
print("(엔트로피가 높음 = 확률이 50:50에 가까움)")
print("="*50)

for rank, idx in enumerate(top_indices, 1):
    idx = int(idx.item())  # <--- [수정] NumPy 값을 안전하게 Python int로 변환

    print(f"\n🥈 Rank {rank}")
    print(f" • 문장: {all_texts[idx]}")
    print(f" • 실제 라벨: {'Noise 2(야민정음)' if all_labels[idx]==1 else 'Noise 1(일반/오타)'}")
    print(f" • 모델 예측: {'Noise 2(야민정음)' if all_preds[idx]==1 else 'Noise 1(일반/오타)'}")
    print(f" • 엔트로피 : {all_entropies[idx]:.4f}")

print("\n✅ 모든 평가 완료.")


📊 엔트로피(Entropy) 기반 평가 시작...
   -> 검증 데이터셋에 대한 예측 및 엔트로피 계산 중...


Evaluating:   0%|          | 0/47 [00:00<?, ?it/s]


🧐 [분석 결과 1] 모델의 확신도(Confidence) 분석
✅ 정답을 맞춘 경우 평균 엔트로피 : 0.0052 (낮을수록 확신함)
❌ 정답을 틀린 경우 평균 엔트로피 : 0.4811 (높을수록 헷갈려함)

👉 해석: 일반적으로 '틀린 경우'의 엔트로피가 '맞춘 경우'보다 높아야 모델이 모르는 것을 모른다고 인식하는 정상적인 상태입니다.

🤯 [분석 결과 2] 모델이 가장 헷갈려한 문장 Top 5
(엔트로피가 높음 = 확률이 50:50에 가까움)

🥈 Rank 1
 • 문장: 오 ~ ~ ~ ~ 아늑 ~ ~ ~ ~ ~ 부산.
 • 실제 라벨: Noise 2(야민정음)
 • 모델 예측: Noise 1(일반/오타)
 • 엔트로피 : 0.6926

🥈 Rank 2
 • 문장: 호텔 3층 " 남풍 "
 • 실제 라벨: Noise 2(야민정음)
 • 모델 예측: Noise 2(야민정음)
 • 엔트로피 : 0.6791

🥈 Rank 3
 • 문장: 비수기라 그런지 좋네요.
 • 실제 라벨: Noise 2(야민정음)
 • 모델 예측: Noise 1(일반/오타)
 • 엔트로피 : 0.6752

🥈 Rank 4
 • 문장: 그리 깨끗하지는 않아요. 에어컨어 좀....
 • 실제 라벨: Noise 1(일반/오타)
 • 모델 예측: Noise 2(야민정음)
 • 엔트로피 : 0.6707

🥈 Rank 5
 • 문장: 바다를 볼 수 있어서 좋음 ~ ~
 • 실제 라벨: Noise 1(일반/오타)
 • 모델 예측: Noise 1(일반/오타)
 • 엔트로피 : 0.6704

✅ 모든 평가 완료.


**직접입력**

In [6]:
# 1. 라이브러리 임포트
import torch
import pandas as pd
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

# GPU 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {device}")

# ==========================================
# 2. 저장된 모델 불러오기 (Load)
# ==========================================
# ★ 중요: 학습 때 저장한 폴더 경로를 정확히 입력해주세요.
SAVE_PATH = "/content/drive/MyDrive/P01_Nandoc/Classifier"

try:
    print(f"🔄 모델 로딩 중... ({SAVE_PATH})")
    tokenizer = ElectraTokenizer.from_pretrained(SAVE_PATH)
    model = ElectraForSequenceClassification.from_pretrained(SAVE_PATH)
    model.to(device)
    model.eval() # 평가 모드로 전환
    print("✅ 모델 로드 완료!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("경로가 정확한지 확인해주세요.")

# ==========================================
# 3. 테스트 데이터 준비 (검증용)
# ==========================================
# 학습 때와 동일하게 데이터를 로드하고 split해서, Validation Set을 Test 용도로 사용합니다.
try:
    path1 = '/content/drive/MyDrive/P01_Nandoc/dataset/일반_10k.csv'
    path2 = '/content/drive/MyDrive/P01_Nandoc/dataset/yamin_11k.csv'
    path3 = '/content/drive/MyDrive/P01_Nandoc/dataset/new_train_yamin_only.csv'
    path4 = '/content/drive/MyDrive/P01_Nandoc/dataset/new_train_normal_only.csv'

    # 데이터 로드
    df1 = pd.read_csv(path1)[['input']].dropna()
    df2 = pd.read_csv(path2)[['input']].dropna()
    df3 = pd.read_csv(path3)[['input']].dropna()
    df4 = pd.read_csv(path4)[['input']].dropna()

    # 라벨링
    df1['label'] = 0
    df4['label'] = 0
    df2['label'] = 1
    df3['label'] = 1

    # Data Re-construction (Train과 동일한 로직)
    df_noise1 = pd.concat([df1, df4])
    if len(df_noise1) > 15000:
        df_noise1 = df_noise1.sample(n=15000, random_state=42).reset_index(drop=True)
    else:
        df_noise1 = df_noise1.reset_index(drop=True)

    df_noise2 = pd.concat([df2, df3])
    if len(df_noise2) > 15000:
        df_noise2 = df_noise2.sample(n=15000, random_state=42).reset_index(drop=True)
    else:
        df_noise2 = df_noise2.reset_index(drop=True)

    # 통합 및 분리
    df_classifier = pd.concat([df_noise1, df_noise2]).sample(frac=1, random_state=42).reset_index(drop=True)
    _, test_df = train_test_split(df_classifier, test_size=0.1, random_state=42)

    print(f"📊 테스트 데이터셋: {len(test_df)}개")

    # 데이터셋 클래스
    class TestDataset(Dataset):
        def __init__(self, df, tokenizer):
            self.df = df
            self.tokenizer = tokenizer
        def __len__(self): return len(self.df)
        def __getitem__(self, idx):
            row = self.df.iloc[idx]
            inputs = self.tokenizer(
                str(row['input']), return_tensors='pt', max_length=128, padding='max_length', truncation=True
            )
            return {
                'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'labels': torch.tensor(row['label'], dtype=torch.long)
            }

    test_loader = DataLoader(TestDataset(test_df, tokenizer), batch_size=64, shuffle=False)

    # ==========================================
    # 4. 전체 성능 평가 (Accuracy)
    # ==========================================
    print("\n📝 전체 정확도 측정 중...")
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"\n🏆 최종 정확도(Accuracy): {acc*100:.2f}%")
    print(classification_report(all_labels, all_preds, target_names=['Noise 1 (오타)', 'Noise 2 (야민정음)']))

except Exception as e:
    print(f"\n⚠️ 데이터 파일이 없거나 오류 발생: {e}")
    print("데이터 평가를 건너뛰고 문장 테스트로 넘어갑니다.")

# ==========================================
# 5. 실전 문장 테스트 (Manual Test)
# ==========================================
def predict(text):
    model.eval()
    inputs = tokenizer(text, return_tensors='pt', truncation=True).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(logits, dim=-1).item()
        prob = torch.softmax(logits, dim=-1)[0][pred].item()

    label_map = {0: "Noise 1 (오타/음운변형)", 1: "Noise 2 (야민정음/시각변형)"}
    return label_map[pred], prob

print("\n============== [개별 문장 테스트] ==============")
samples = [
    "바베큐장 요금을 내고 이용했는데 경말 ㅌ로 옆 좁은 공간에 억겨로 만든 공간이었어8. 좁아서 한쪽 의자= 71울어진 언덕에 두ヱ 겨윽 앉았어8. 새벽 늦게까거 바ro 옆 도로O||서 과속하는 차dl 큰 소m이 너무 잘 8려서 잠을 잘 못 잤어요. ",        # Noise 1 예상
    "go성네ㅋr Oㅏ주 좋슴니다. 윌치E 2동ㅎr긴 편Ha꾸yO. 빠료 근쳐엔 버^ 정류자0I e써오. 간댠한 초씩도 있어서 oㅑ침에 ㅂrro e용ㅎr거 편했슴니댜.네넴띤도 먹었어얌ㅋㅋ 저권분8 다 친절하셜쎠8~ 편안한게 잘 찌냈습niㅁ-.",       # Noise 2 예상
    "싀당2 너ㅁ 조용해서 한입 물 때파ㅁ- 솓21나는 듯한 경성O1 느껴거는데, 주ㅊr는 완전 난R1OF. 지Ha 6층O|| 즉ㅊr가 꽉 ㅊr서 밖O||서 대기hooㅑ 하는데, 20분OI 저났=데ㅌ oㅕ전ㅎl 주ㅊr 못Ha끄? 결국 ㄹr운지에서 커괴 한 잔 마ㅅl펴 ㅅl간을 보LㅐOㅑ 했어. 가격은 B싸긴 한데, 메뉴= 신선ㅎrヱ 맛OI 일품2어서 조금은 감수할 만한 수준이긴 ho. ㅁ-만, ㅁ-m엔 주ㅊr 문제 좀 더 해결ㅎri 즉면 좋겠어!",      # Noise 2 (머=대)
    "ㄱr성비가 너무 좋아요 여기 다시 오고 싶어요",      # Noise 2 (네넴띤=비빔면)
    "오늘do Ol ㅋr페는 진짜 힐링 존! 창Lr에서 보= ㅂr㉰ 전망O1 팬날보ㅁ- 예뻐서 눈e 번쩍. Oㅏ침싀사do 푸짐Ha그 신선한 JH료go 듬뿍 dl어7r 있어서 한입O|| 행복이 머져. 직원분8 서13^E 10점O|| 딱 10점이2r서 따음OI 편안했어. ㅁ-m엔 꼭 친ヲ랑 같ㅇl 오ㅈr! ㅎㅎ", # Noise 1 예상
    "겨난즉O|| ㉠r족마 함께 OI 호텔O|| 3박했는데, Lr격2 10만원O|| 4박0lㄹr 7r성네go 완전 떠박e었어dD. 체크인부터 친절ㅎrgay 맞Oㅏ줘서 ㅂr2o 편안함을 느꼈죠. 방은 넓끄 침머7r 푹신ㅎri서 ㅈr존감2 200% 올ㄹr갔어8. 특ㅎl Oㅏ침 뷔페7r 2만원O|| 푸짐했끄, 긲끠= ㅁ료ro 제끙EH서 너ㅁ 좋OㅏdD. 바쁜 일경 중O||do oㅕ윾를 느낄 수 있었으ni ㉰m엔 꼭 ㅁ-ㅅl gooㅑ겠어dD!"       # Noise 2 (댕댕이=멍멍이)
]

for s in samples:
    res, conf = predict(s)
    print(f"입력: {s}")
    print(f"판단: {res} (확신도: {conf*100:.1f}%)")
    print("-" * 30)

✅ Device: cuda
🔄 모델 로딩 중... (/content/drive/MyDrive/P01_Nandoc/Classifier)
✅ 모델 로드 완료!
📊 테스트 데이터셋: 3000개

📝 전체 정확도 측정 중...


Testing:   0%|          | 0/47 [00:00<?, ?it/s]


🏆 최종 정확도(Accuracy): 99.30%
                precision    recall  f1-score   support

  Noise 1 (오타)       0.99      0.99      0.99      1476
Noise 2 (야민정음)       0.99      0.99      0.99      1524

      accuracy                           0.99      3000
     macro avg       0.99      0.99      0.99      3000
  weighted avg       0.99      0.99      0.99      3000


============== [개별 문장 테스트] ==============
입력: 바베큐장 요금을 내고 이용했는데 경말 ㅌ로 옆 좁은 공간에 억겨로 만든 공간이었어8. 좁아서 한쪽 의자= 71울어진 언덕에 두ヱ 겨윽 앉았어8. 새벽 늦게까거 바ro 옆 도로O||서 과속하는 차dl 큰 소m이 너무 잘 8려서 잠을 잘 못 잤어요. 
판단: Noise 2 (야민정음/시각변형) (확신도: 100.0%)
------------------------------
입력: go성네ㅋr Oㅏ주 좋슴니다. 윌치E 2동ㅎr긴 편Ha꾸yO. 빠료 근쳐엔 버^ 정류자0I e써오. 간댠한 초씩도 있어서 oㅑ침에 ㅂrro e용ㅎr거 편했슴니댜.네넴띤도 먹었어얌ㅋㅋ 저권분8 다 친절하셜쎠8~ 편안한게 잘 찌냈습niㅁ-.
판단: Noise 2 (야민정음/시각변형) (확신도: 100.0%)
------------------------------
입력: 싀당2 너ㅁ 조용해서 한입 물 때파ㅁ- 솓21나는 듯한 경성O1 느껴거는데, 주ㅊr는 완전 난R1OF. 지Ha 6층O|| 즉ㅊr가 꽉 ㅊr서 밖O||서 대기hooㅑ 하는데, 20분OI 저났=데ㅌ oㅕ전ㅎl 주ㅊr 못Ha끄? 결국 ㄹr운지에서 커괴 한 잔 마ㅅl펴 ㅅl간을 보LㅐOㅑ 했어. 가격은 B싸